In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=35816a97290f043429ce7d0dedc414258de14176429caa3acc79b72c20a04112
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

#### Confirm CUDA

In [ ]:
import torch
torch.cuda.is_available()

True

# Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/detoxification_dataset.csv", split = "train[400000:500000]")

In [ ]:
dataset

Dataset({
    features: ['toxic', 'detoxified'],
    num_rows: 100000
})

In [ ]:
data = dataset.train_test_split(test_size=0.01, seed=42)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['toxic', 'detoxified'],
        num_rows: 99000
    })
    test: Dataset({
        features: ['toxic', 'detoxified'],
        num_rows: 1000
    })
})

In [ ]:
import pandas as pd
test_df = pd.DataFrame()
test_df["toxic_texts"] = data["test"]["toxic"]
test_df

,toxic_texts
0,I don't give a shit.
1,I've never seen anything more disgusting in my...
2,my biggest client is a crook.
3,"When that o'malley kid cheated on you, - I was..."
4,"Those morons. if escaping was easy, I would ha..."
...,...
995,Let's get the fuck out ofhere!
996,"shut up, Bordin. Or I'll punish you."
997,We are not goanna kill a goatin this man's bac...
998,They wanted to see what screwing a dead man fe...


# Load quantized model

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "/content/bloom-detoxification/checkpoint-800"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
qa_model = PeftModel.from_pretrained(model, peft_model_id)

# Inference on the loaded model

In [ ]:
def make_inference(toxic: str) -> str:
  batch = tokenizer(f"### Detoxify:\n{toxic}\n\n### detoxified:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True).split("\n")[-1]

In [ ]:
generated = []
detox_actual = []
inputs = []

count = 0
for text in data["test"]:
  pred = make_inference(text["toxic"])
  inputs.append(text["toxic"])
  detox_actual.append(text["detoxified"])
  generated.append(pred)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
inputs[0], detox_actual[0], generated[0]

("I don't give a shit.", "I don't care.", "I don't care.")

In [ ]:
len(inputs), len(detox_actual), len(generated)

(1000, 1000, 1000)

In [ ]:
import pandas as pd
test_df = pd.DataFrame()
test_df["input"] = inputs
test_df["actual"] = detox_actual
test_df["generated"] = generated

# Rouge scores for the bloom-3b 8bit quantized model

In [ ]:
from rouge_score import rouge_scorer
import pandas as pd

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for index, row in test_df.iterrows():
    reference = row['actual']
    prediction = row['generated']
    scores = scorer.score(reference, prediction)

    rouge_scores.append(scores)
rouge_df = pd.DataFrame(rouge_scores)

In [ ]:
rouge1 = list(rouge_df["rouge1"])
rouge2 = list(rouge_df["rouge2"])
rougeL = list(rouge_df["rougeL"])

def mean_rouge(ls : list):
    sum = 0
    count = 0
    for scores in ls:
        count+=1
        sum+=scores[-1]
    return sum/count
avg_rouge1 = mean_rouge(rouge1)
avg_rouge2 = mean_rouge(rouge2)
avg_rougeL = mean_rouge(rougeL)

In [ ]:
avg_rouge1, avg_rouge2, avg_rougeL

(0.5922136553827577, 0.3568738760624405, 0.5721437267178481)

# BERT embedding similarity for the bloom-3b 8bit quantized model

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import warnings

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to("cuda")

def bert_text_similarity(reference, candidate, tokenizer, model):
    tokens_ref = tokenizer.encode(reference, add_special_tokens=True, return_tensors="pt").to("cuda")
    tokens_cand = tokenizer.encode(candidate, add_special_tokens=True, return_tensors="pt").to("cuda")

    with torch.no_grad():
        embeddings_ref = model(tokens_ref)[0][:, 0, :]
        embeddings_cand = model(tokens_cand)[0][:, 0, :]

    similarity_score = cosine_similarity(embeddings_ref.cpu(), embeddings_cand.cpu())[0][0]

    return similarity_score

def calculate_mean_similarity(df, tokenizer, model):
    total_similarity = 0.0
    num_pairs = len(df)

    for index, row in df.iterrows():
        human_response = row["actual"]
        model_response = row["generated"]

        similarity_score = bert_text_similarity(human_response, model_response, tokenizer, model)
        total_similarity += similarity_score

    mean_similarity = total_similarity / num_pairs
    return mean_similarity

mean_similarity = calculate_mean_similarity(test_df, tokenizer, model)
print(f"Mean BERT Text Similarity Score: {mean_similarity:.2f}")

Mean BERT Text Similarity Score: 0.94
